#  trail run estimate

at the trail, it is especially important to correctly asses duration of the activitity, since it is a safety factor.

the goal here is to asses correctly the trail run time, point out potentially too steep regions, snow conditions, need for light, view points

In [ ]:
#logging.basicConfig(level=logging.DEBUG)

from importlib import reload  # Not needed in Python 2
import logging
reload(logging)

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [ ]:
%matplotlib notebook
import matplotlib as mpl
from matplotlib import pylab as plt
from matplotlib import cm as cm
import numpy as np
import requests
import time
from IPython.display import display, HTML
from urllib.parse import urlencode
import yaml

In [ ]:
c = yaml.safe_load(open("strava-client.yaml"))

client_id = c['client_id']
client_secret = c['client_secret']

In [ ]:
# auth?

print("http://www.strava.com/oauth/authorize?" + urlencode(dict(
        client_id=client_id,
        response_type="code",
        redirect_uri="http://localhost/exchange_token",
        approval_prompt="force",
        scope="activity:read_all",
    )))

code=input()

r=requests.post("https://www.strava.com/api/v3/oauth/token",
  data=dict(
      client_id=client_id,
      client_secret=client_secret,
      code=code,
      grant_type="authorization_code",
  )
).json()

token=r['access_token']

In [ ]:
token=r['access_token']

requests.get("https://www.strava.com/api/v3/athlete", headers={"Authorization": f"Bearer {token}"}).json()

In [ ]:
import time
import bravado
import bravado.client
import bravado.requests_client
import bravado.http_client

reload(bravado), reload(bravado.client), reload(bravado.requests_client), reload(bravado.http_client)

from bravado.requests_client import RequestsClient
from bravado.client import SwaggerClient

http_client = RequestsClient()
http_client.set_api_key(
    'www.strava.com', 'Bearer '+token,
    param_name='Authorization', param_in='header'
)

client = SwaggerClient.from_url(
    'https://developers.strava.com/swagger/swagger.json',
    http_client=http_client,
    config={
        'validate_swagger_spec': False,
        'validate_responses': False,
        'validate_requests': False,
    },
)
athlete = client.Athletes.getLoggedInAthlete().response().result
athlete

In [ ]:
routes = requests.get(f"https://www.strava.com/api/v3/athletes/{athlete['id']}/routes", 
                              params=dict(
                                  per_page=100
                              ),
                              headers={'Authorization': 'Bearer '+token}
                             ).json()

In [ ]:
import io
import gpxpy

route = routes[5]

print("getting route gpx for", route['id'])
route_gpx = gpxpy.parse(io.BytesIO(requests.get(f"https://www.strava.com/api/v3//routes/{route['id']}/export_gpx", 
                              params=dict(
                                  per_page=100
                              ),
                              headers={'Authorization': 'Bearer '+token}
                             ).content))
route_points = route_gpx.tracks[0].segments[0].points

print("got", route_gpx.tracks[0])

In [ ]:
d_route_d3 = [0] + [a.distance_3d(b) for a,b in zip(route_points[:-1], route_points[1:])]  
route_d3 = np.cumsum(d_route_d3)

route_lat = np.array([p.latitude for p in route_points])
route_lon = np.array([p.longitude for p in route_points])
route_altitude = np.array([p.elevation for p in route_points])
route_grade = d_(route_altitude)/d_(route_d3, 10)*100.

plt.figure()
    
plt.scatter(
    route_lon,
    route_lat,
    c=cm.jet(route_grade/50.),
)

plt.grid()
#plt.ylim([-50, 50])

In [ ]:
p.longitude

In [ ]:
if False:
    activities = client.Activities.getLoggedInAthleteActivities(
        after=int(time.time() - 720*24*3600),
        perPage=100,
    ).response().result
else:
    activities = requests.get("https://www.strava.com/api/v3/athlete/activities", 
                              params=dict(
                                  per_page=100
                              ),
                              headers={'Authorization': 'Bearer '+token}
                             ).json()

In [ ]:
for activity in activities:
    if 'Ride' in activity['name'] or activity['total_elevation_gain'] < 300:
        activity.pop('streams', None)
        continue
        
        
    
    activity['streams'] = client.Streams.getActivityStreams(
        id=activity['id'], 
        keys="time, distance, latlng, altitude, velocity_smooth, heartrate, cadence, watts, temp, moving, grade_smooth".split(", "),
        key_by_type=True,
    ).response().result


    
    print(activity['name'])

In [ ]:
def d_(x, n=10):
    d_filter = np.zeros(n)
    d_filter[0] = 1
    d_filter[-1] = -1
    return np.convolve(x, d_filter, mode='same')

In [ ]:
#plt.figure()

for activity in activities[2:]:
    if 'streams' not in activity: continue
    streams = activity['streams']
    
    
    distance = np.array(streams['distance']['data'])
    altitude = np.array(streams['altitude']['data'])
    ttime = np.array(streams['time']['data'])
    latlng = np.array(streams['latlng']['data'])


    d_N = 10
    
    d_distance = d_(distance, d_N)
    d_altitude = d_(altitude, d_N)
    d_d3 = (d_distance**2 + d_altitude**2)**0.5
    d_time = d_(ttime, d_N)
    
    grade = d_altitude/d_d3*100.

    def dealias(x):
        mn = np.round(np.abs(x[x>0]).min(), 5)
        print("found step", mn)
        x += np.random.rand(altitude.shape[0])*mn - mn/2.

    dealias(d_altitude)
    dealias(d_distance)

    m = np.abs(d_altitude)<1000

    def p2d(x,y, x_bins, y_bins, mode='hist'):

        if mode == "scatter":
            plt.scatter(x,y,s=5)
        elif mode == 'hist':
            return plt.hist2d(x,y, bins=(
                    x_bins,
                    y_bins,
                )
            )
        elif mode == 'contour':
            h2, a,b  = np.histogram2d(x,y, bins=(
                    x_bins,
                    y_bins,
                )
            )

            plt.contourf(
                a[:-1],
                b[:-1],
                np.transpose(h2),
                #levels=np.logspace(np.log10(h2.max()/1000.), np.log10(h2.max()), 100)
                levels=np.linspace(0, h2.max(), 100)
            )


    for mode in 'hist',: #'contour':
        f, (ax1,ax2)= plt.subplots(1,2,figsize=(9,5))
        lut_speed_grade = p2d(
            d_d3[m]/d_time[m]*3600./1000.,
            #d_altitude[m]/d_time[m]*3600.,
            #60./(d_distance[m]/d_time[m]*3600./1000.),
            #d_distance[m]/d_time[m]*3600./1000.,
            #d_altitude[m]/d_time[m]*3600.,
            #d_altitude[m]/d_time[m]*3600,
            d_altitude[m]/d_d3[m]*100,
            mode = mode,
            x_bins = np.linspace(0, 15, 70),
            y_bins = np.linspace(-60, 60, 70),
            #y_bins = np.linspace(-2000, 2000, 70),
        )
        
        plt.subplots_adjust(wspace=0)
        
        ax1.scatter(latlng[:,1], latlng[:,0], 
            #c=cm.jet(np.array(streams['velocity_smooth']['data'])/6.),
            c=cm.jet((
                np.array(d_altitude*3600)/(1000.)
                #np.array(streams['altitude']['data'])-320)/(600-320)
            ),
        ))
        
        plt.title(activity['name'])
        
    print("total time", (ttime[-1] - ttime[0])/3600., "hr")
    
    x_grade = np.linspace(-50,50)
    ax2.plot(
        [speed_estim_for_grade(x, lut_speed_grade) for x in x_grade],
        x_grade,
    )
    
    total_time_estim = 0
    sum_time = 0
    i = 0
    for d_d, d_a, d_t, _t in zip(d_d3, d_altitude, d_time, ttime):
        i += 1
        if i<30: continue
        if i>len(d_time)-100: continue
        
        #print(d_d, d_a)
        
        grade = d_a/d_d*100.
        
        d_time_estim = d_d/(speed_estim_for_grade(grade, lut_speed_grade)/3600.*1000)
        
        total_time_estim += d_time_estim
        sum_time += d_t
                        
        print("since", _t - ttime[0], "estim", total_time_estim, "sum", sum_time)
        print("grade", grade, "speed", d_d/d_t*3.6, "estim speed", speed_estim_for_grade(grade, lut_speed_grade), "time estim", d_time_estim, "time spent", d_t)
        
        
        
    print("total time estimate", total_time_estim/3600., "hr", sum_time/3600.)
        
    break

    #ax=plt.gca()
    #ax2=plt.twiny()
    #ax2.set_xlim(60./np.array(ax.get_xlim()))

    #plt.xlabel("pace")
    #plt.ylabel("grade")
    #plt.ylabel("VAM")

In [ ]:

def speed_estim_for_grade(grade, lut_speed_grade):

    i_grade = np.abs(grade - lut_speed_grade[2][:-1]).argmin()

    dp = lut_speed_grade[0][:,i_grade]
    

    return np.sum(lut_speed_grade[1][:-1]*dp)/np.sum(dp)

    #lut_speed_grade[2][i_grade], grade, lut_speed_grade[0][i_grade]
    
speed_estim_for_grade(10, lut_speed_grade)

In [ ]:
plt.figure()

plt.hist(d_altitude[m]/d_time[m]*3600, 300)